#fillna -1

In [1]:
# -*- coding:utf-8 -*-
%cd E:\竞赛\atec
import numpy as np
import pandas as pd

E:\竞赛\atec


In [2]:
NUM_DATASET = 10
cweight = 1
val_size = 1000
rate = 60

In [3]:
data = pd.read_csv("atec_anti_fraud_train.csv")

In [4]:
import lightgbm as lgb

In [5]:
data.fillna(-1,inplace=True)

In [6]:
data["label"] = data.label.replace(-1,1)

In [7]:
one = data[data.label==1]
zero = data[data.label==0]

In [8]:
from sklearn.model_selection import train_test_split

In [9]:
one_train,one_val = train_test_split(one,test_size=val_size,stratify=one.date,random_state=42)

In [10]:
train_size = one_train.shape[0]

In [11]:
zeroset = []
for i in range(NUM_DATASET):
    _,zero_tmp = train_test_split(zero,test_size=train_size,stratify=zero.date,random_state=i)
    zeroset.append(zero_tmp)

In [12]:
dataset = []
for i in range(NUM_DATASET):
    dataset_tmp = pd.concat([zeroset[i],one_train],axis=0)
    dataset_tmp["weight"] = dataset_tmp.label.apply(lambda x:cweight if x==1 else 1)
    dataset.append(dataset_tmp)

In [13]:
_,zero_val= train_test_split(zero,test_size=val_size*rate,stratify=zero.date,random_state=42)
val = pd.concat([zero_val,one_val],axis=0)

{'learning_rate': 0.11603433815528452, 'num_leaves': 811, 'max_depth': 98, 'max_bin': 780, 'min_data_in_leaf': 5, 'min_sum_hessian_in_leaf': 0.0028446154613853292, 'feature_fraction': 0.4801070772666266}

In [14]:
params = {
    "task": "train",
    "boosting_type": "gbdt",
    "objective": "binary",
    "metric": "auc",
    "learning_rate": 0.1,
    "num_leaves": 811,
    "max_depth": 98,
    "bagging_fraction": 0.95,
    "bagging_freq": 5,
    "feature_fraction": 0.5,
    "max_bin":780,
    "min_data_in_leaf ":5,
    "min_sum_hessian_in_leaf":0.00285,
    "min_gain_to_split":0.0,
    "lambda_l1":0.0,
    "lambda_l2":0.0
}
#"subsample_for_bin": 1000000,

In [15]:
tmp = dataset[0]
tmp[tmp.label==1].weight.head()

306881    1
686705    1
569569    1
330690    1
746297    1
Name: weight, dtype: int64

In [16]:
lgbmodellist = []
for i in range(NUM_DATASET):
    print(i)
    data_tmp = dataset[i]
    d_train = lgb.Dataset(data_tmp[data_tmp.columns.difference(["id","label","weight","date"])],label=data_tmp.label,weight=data_tmp.weight)
    d_val = lgb.Dataset(val[val.columns.difference(["id","label","weight","date"])],label=val.label,reference=d_train)#,weight=val.weight
    bst = lgb.train(params,d_train,500,early_stopping_rounds=10,valid_sets=[d_train,d_val])#
    lgbmodellist.append(bst)

0
[1]	training's auc: 0.97901	valid_1's auc: 0.953878
Training until validation scores don't improve for 10 rounds.
[2]	training's auc: 0.985793	valid_1's auc: 0.960043
[3]	training's auc: 0.987861	valid_1's auc: 0.965201
[4]	training's auc: 0.990626	valid_1's auc: 0.971197
[5]	training's auc: 0.991566	valid_1's auc: 0.972275
[6]	training's auc: 0.992927	valid_1's auc: 0.97411
[7]	training's auc: 0.993451	valid_1's auc: 0.974228
[8]	training's auc: 0.994016	valid_1's auc: 0.974575
[9]	training's auc: 0.994374	valid_1's auc: 0.975015
[10]	training's auc: 0.994693	valid_1's auc: 0.975072
[11]	training's auc: 0.995256	valid_1's auc: 0.976236
[12]	training's auc: 0.995643	valid_1's auc: 0.976874
[13]	training's auc: 0.995974	valid_1's auc: 0.977129
[14]	training's auc: 0.996302	valid_1's auc: 0.977311
[15]	training's auc: 0.996611	valid_1's auc: 0.977942
[16]	training's auc: 0.996907	valid_1's auc: 0.978195
[17]	training's auc: 0.997179	valid_1's auc: 0.978068
[18]	training's auc: 0.997453

2
[1]	training's auc: 0.978503	valid_1's auc: 0.950372
Training until validation scores don't improve for 10 rounds.
[2]	training's auc: 0.985365	valid_1's auc: 0.959126
[3]	training's auc: 0.987971	valid_1's auc: 0.965075
[4]	training's auc: 0.990728	valid_1's auc: 0.969956
[5]	training's auc: 0.991662	valid_1's auc: 0.971144
[6]	training's auc: 0.992882	valid_1's auc: 0.973176
[7]	training's auc: 0.993374	valid_1's auc: 0.97351
[8]	training's auc: 0.993949	valid_1's auc: 0.973704
[9]	training's auc: 0.994359	valid_1's auc: 0.973839
[10]	training's auc: 0.994681	valid_1's auc: 0.974216
[11]	training's auc: 0.995207	valid_1's auc: 0.97523
[12]	training's auc: 0.995649	valid_1's auc: 0.976266
[13]	training's auc: 0.995964	valid_1's auc: 0.976081
[14]	training's auc: 0.996254	valid_1's auc: 0.976436
[15]	training's auc: 0.996594	valid_1's auc: 0.976782
[16]	training's auc: 0.996918	valid_1's auc: 0.977146
[17]	training's auc: 0.997175	valid_1's auc: 0.977166
[18]	training's auc: 0.997424

4
[1]	training's auc: 0.979795	valid_1's auc: 0.952612
Training until validation scores don't improve for 10 rounds.
[2]	training's auc: 0.98577	valid_1's auc: 0.961017
[3]	training's auc: 0.988452	valid_1's auc: 0.966534
[4]	training's auc: 0.991043	valid_1's auc: 0.970535
[5]	training's auc: 0.991967	valid_1's auc: 0.97193
[6]	training's auc: 0.993197	valid_1's auc: 0.974089
[7]	training's auc: 0.993594	valid_1's auc: 0.974695
[8]	training's auc: 0.994149	valid_1's auc: 0.975261
[9]	training's auc: 0.9945	valid_1's auc: 0.975791
[10]	training's auc: 0.994852	valid_1's auc: 0.975936
[11]	training's auc: 0.995384	valid_1's auc: 0.976964
[12]	training's auc: 0.995838	valid_1's auc: 0.977368
[13]	training's auc: 0.996139	valid_1's auc: 0.977186
[14]	training's auc: 0.996425	valid_1's auc: 0.977545
[15]	training's auc: 0.996725	valid_1's auc: 0.977959
[16]	training's auc: 0.997068	valid_1's auc: 0.978331
[17]	training's auc: 0.997317	valid_1's auc: 0.978375
[18]	training's auc: 0.997558	v

[65]	training's auc: 1	valid_1's auc: 0.98249
[66]	training's auc: 1	valid_1's auc: 0.982563
[67]	training's auc: 1	valid_1's auc: 0.982582
[68]	training's auc: 1	valid_1's auc: 0.98264
[69]	training's auc: 1	valid_1's auc: 0.982676
[70]	training's auc: 1	valid_1's auc: 0.98266
[71]	training's auc: 1	valid_1's auc: 0.982662
[72]	training's auc: 1	valid_1's auc: 0.982651
[73]	training's auc: 1	valid_1's auc: 0.982638
[74]	training's auc: 1	valid_1's auc: 0.982666
[75]	training's auc: 1	valid_1's auc: 0.982671
[76]	training's auc: 1	valid_1's auc: 0.982739
[77]	training's auc: 1	valid_1's auc: 0.982789
[78]	training's auc: 1	valid_1's auc: 0.982811
[79]	training's auc: 1	valid_1's auc: 0.982838
Early stopping, best iteration is:
[69]	training's auc: 1	valid_1's auc: 0.982676
6
[1]	training's auc: 0.979454	valid_1's auc: 0.951343
Training until validation scores don't improve for 10 rounds.
[2]	training's auc: 0.985967	valid_1's auc: 0.959878
[3]	training's auc: 0.988238	valid_1's auc: 0.

[65]	training's auc: 1	valid_1's auc: 0.982056
[66]	training's auc: 1	valid_1's auc: 0.98209
[67]	training's auc: 1	valid_1's auc: 0.982116
[68]	training's auc: 1	valid_1's auc: 0.982108
[69]	training's auc: 1	valid_1's auc: 0.982122
[70]	training's auc: 1	valid_1's auc: 0.982096
[71]	training's auc: 1	valid_1's auc: 0.982114
[72]	training's auc: 1	valid_1's auc: 0.982115
[73]	training's auc: 1	valid_1's auc: 0.982146
[74]	training's auc: 1	valid_1's auc: 0.982132
[75]	training's auc: 1	valid_1's auc: 0.982117
Early stopping, best iteration is:
[65]	training's auc: 1	valid_1's auc: 0.982056
8
[1]	training's auc: 0.980068	valid_1's auc: 0.950721
Training until validation scores don't improve for 10 rounds.
[2]	training's auc: 0.986431	valid_1's auc: 0.960724
[3]	training's auc: 0.98892	valid_1's auc: 0.965378
[4]	training's auc: 0.991239	valid_1's auc: 0.970389
[5]	training's auc: 0.992154	valid_1's auc: 0.97122
[6]	training's auc: 0.9933	valid_1's auc: 0.973113
[7]	training's auc: 0.99

[47]	training's auc: 0.999995	valid_1's auc: 0.982414
[48]	training's auc: 0.999996	valid_1's auc: 0.982511
[49]	training's auc: 0.999997	valid_1's auc: 0.982503
[50]	training's auc: 0.999998	valid_1's auc: 0.982549
[51]	training's auc: 0.999999	valid_1's auc: 0.982611
[52]	training's auc: 0.999999	valid_1's auc: 0.98265
[53]	training's auc: 1	valid_1's auc: 0.982717
[54]	training's auc: 1	valid_1's auc: 0.982784
[55]	training's auc: 1	valid_1's auc: 0.982794
[56]	training's auc: 1	valid_1's auc: 0.982705
[57]	training's auc: 1	valid_1's auc: 0.982714
[58]	training's auc: 1	valid_1's auc: 0.982814
[59]	training's auc: 1	valid_1's auc: 0.982816
[60]	training's auc: 1	valid_1's auc: 0.982748
[61]	training's auc: 1	valid_1's auc: 0.982824
[62]	training's auc: 1	valid_1's auc: 0.982808
[63]	training's auc: 1	valid_1's auc: 0.982884
[64]	training's auc: 1	valid_1's auc: 0.982918
[65]	training's auc: 1	valid_1's auc: 0.982947
[66]	training's auc: 1	valid_1's auc: 0.982983
[67]	training's auc

# Validation

In [17]:
from sklearn.metrics import roc_curve,roc_auc_score
def score_atc(y_true,y_pred_prob):
    fpr, tpr, thresholds = roc_curve(y_true,y_pred_prob, pos_label=1,drop_intermediate=False)
    score=0.4*tpr[(fpr>=0.001) & (fpr<=0.0011)][0]+0.3*tpr[(fpr>=0.005) & (fpr<=0.0051)][0]+0.3*tpr[(fpr>=0.01) & (fpr<=0.011)][0]
    return score

In [18]:
res = []
for i in range(NUM_DATASET):
    model = lgbmodellist[i]
    pred = model.predict(val[val.columns.difference(["id","label","weight","date"])],num_iteration = model.best_iteration)
    res.append(pred)
result = np.array(res)
result = result.mean(axis=0)

In [19]:
print("class weight:",cweight,"num of dataset:",NUM_DATASET,"val size:",val_size,"one zero rate:",rate)
print(params)
print(score_atc(val.label,result))
print(roc_auc_score(val.label,result))

class weight: 1 num of dataset: 10 val size: 1000 one zero rate: 60
{'task': 'train', 'boosting_type': 'gbdt', 'objective': 'binary', 'metric': 'auc', 'learning_rate': 0.1, 'num_leaves': 811, 'max_depth': 98, 'bagging_fraction': 0.95, 'bagging_freq': 5, 'feature_fraction': 0.5, 'max_bin': 780, 'min_data_in_leaf ': 5, 'min_sum_hessian_in_leaf': 0.00285, 'min_gain_to_split': 0.0, 'lambda_l1': 0.0, 'lambda_l2': 0.0, 'verbose': 1}
0.4615
0.9847927000000001


val without weight
[87]	training's auc: 1	valid_1's auc: 0.981098
[85]	training's auc: 1	valid_1's auc: 0.98215
[95]	training's auc: 1	valid_1's auc: 0.981944
class weight: 100 num of dataset: 10 val size: 1000 one zero rate: 60
{'task': 'train', 'boosting_type': 'gbdt', 'objective': 'binary', 'metric': 'auc', 'learning_rate': 0.1, 'num_leaves': 811, 'max_depth': 98, 'bagging_fraction': 1, 'bagging_freq': 1, 'feature_fraction': 0.5, 'max_bin': 780, 'min_data_in_leaf ': 5, 'min_sum_hessian_in_leaf': 0.00285, 'min_gain_to_split': 0.0, 'lambda_l1': 0.0, 'lambda_l2': 0.0, 'verbose': 1}
0.46299999999999997
0.9845535666666667

# 新参数

val without weight
[91]	training's auc: 1	valid_1's auc: 0.982884
[88]	training's auc: 1	valid_1's auc: 0.983266
[86]	training's auc: 1	valid_1's auc: 0.982375
class weight: 1 num of dataset: 10 val size: 1000 one zero rate: 60
{'task': 'train', 'boosting_type': 'gbdt', 'objective': 'binary', 'metric': 'auc', 'learning_rate': 0.1, 'num_leaves': 280, 'max_depth': 20, 'bagging_fraction': 1, 'bagging_freq': 1, 'feature_fraction': 1, 'max_bin': 200, 'min_data_in_leaf ': 100, 'min_sum_hessian_in_leaf': 0.0028, 'min_gain_to_split': 0.0, 'lambda_l1': 0.0, 'lambda_l2': 0.0, 'verbose': 1}
0.44889999999999997
0.9850727999999999

val without weight
[95]	training's auc: 1	valid_1's auc: 0.982679
[105]	training's auc: 1	valid_1's auc: 0.982977
[98]	training's auc: 1	valid_1's auc: 0.98231
class weight: 10 num of dataset: 10 val size: 1000 one zero rate: 60
{'task': 'train', 'boosting_type': 'gbdt', 'objective': 'binary', 'metric': 'auc', 'learning_rate': 0.1, 'num_leaves': 280, 'max_depth': 20, 'bagging_fraction': 1, 'bagging_freq': 1, 'feature_fraction': 1, 'max_bin': 200, 'min_data_in_leaf ': 100, 'min_sum_hessian_in_leaf': 0.0028, 'min_gain_to_split': 0.0, 'lambda_l1': 0.0, 'lambda_l2': 0.0, 'verbose': 1}
0.4633
0.9853992333333333

val without weight
[108]	training's auc: 1	valid_1's auc: 0.981225
[118]	training's auc: 1	valid_1's auc: 0.983014
[114]	training's auc: 1	valid_1's auc: 0.982188
class weight: 50 num of dataset: 10 val size: 1000 one zero rate: 60
{'task': 'train', 'boosting_type': 'gbdt', 'objective': 'binary', 'metric': 'auc', 'learning_rate': 0.1, 'num_leaves': 280, 'max_depth': 20, 'bagging_fraction': 1, 'bagging_freq': 1, 'feature_fraction': 1, 'max_bin': 200, 'min_data_in_leaf ': 100, 'min_sum_hessian_in_leaf': 0.0028, 'min_gain_to_split': 0.0, 'lambda_l1': 0.0, 'lambda_l2': 0.0, 'verbose': 1}
0.4658
0.9851268666666667

"submission62602.csv"
val without weight
[117]	training's auc: 1	valid_1's auc: 0.981632
[119]	training's auc: 0.999996	valid_1's auc: 0.982656
[122]	training's auc: 1	valid_1's auc: 0.982106
class weight: 100 num of dataset: 10 val size: 1000 one zero rate: 60
{'task': 'train', 'boosting_type': 'gbdt', 'objective': 'binary', 'metric': 'auc', 'learning_rate': 0.1, 'num_leaves': 280, 'max_depth': 20, 'bagging_fraction': 1, 'bagging_freq': 1, 'feature_fraction': 1, 'max_bin': 200, 'min_data_in_leaf ': 100, 'min_sum_hessian_in_leaf': 0.0028, 'min_gain_to_split': 0.0, 'lambda_l1': 0.0, 'lambda_l2': 0.0, 'verbose': 1}
0.4661
0.9851012333333334

val without weight
[116]	training's auc: 1	valid_1's auc: 0.981018
[113]	training's auc: 0.999964	valid_1's auc: 0.981058
[129]	training's auc: 1	valid_1's auc: 0.981054
class weight: 200 num of dataset: 10 val size: 1000 one zero rate: 60
{'task': 'train', 'boosting_type': 'gbdt', 'objective': 'binary', 'metric': 'auc', 'learning_rate': 0.1, 'num_leaves': 280, 'max_depth': 20, 'bagging_fraction': 1, 'bagging_freq': 1, 'feature_fraction': 1, 'max_bin': 200, 'min_data_in_leaf ': 100, 'min_sum_hessian_in_leaf': 0.0028, 'min_gain_to_split': 0.0, 'lambda_l1': 0.0, 'lambda_l2': 0.0, 'verbose': 1}
0.4384
0.9833676833333334

val without weight
[108]	training's auc: 0.999703	valid_1's auc: 0.977847
[116]	training's auc: 0.999911	valid_1's auc: 0.978548
[96]	training's auc: 0.997554	valid_1's auc: 0.978073
class weight: 1000 num of dataset: 10 val size: 1000 one zero rate: 60
{'task': 'train', 'boosting_type': 'gbdt', 'objective': 'binary', 'metric': 'auc', 'learning_rate': 0.1, 'num_leaves': 280, 'max_depth': 20, 'bagging_fraction': 1, 'bagging_freq': 1, 'feature_fraction': 1, 'max_bin': 200, 'min_data_in_leaf ': 100, 'min_sum_hessian_in_leaf': 0.0028, 'min_gain_to_split': 0.0, 'lambda_l1': 0.0, 'lambda_l2': 0.0, 'verbose': 1}
0.4295
0.9815078

val without weight
[67]	training's auc: 0.980262	valid_1's auc: 0.968273
[86]	training's auc: 0.987735	valid_1's auc: 0.972085
[161]	training's auc: 1	valid_1's auc: 0.975383
class weight: 10000 num of dataset: 10 val size: 1000 one zero rate: 60
{'task': 'train', 'boosting_type': 'gbdt', 'objective': 'binary', 'metric': 'auc', 'learning_rate': 0.1, 'num_leaves': 280, 'max_depth': 20, 'bagging_fraction': 1, 'bagging_freq': 1, 'feature_fraction': 1, 'max_bin': 200, 'min_data_in_leaf ': 100, 'min_sum_hessian_in_leaf': 0.0028, 'min_gain_to_split': 0.0, 'lambda_l1': 0.0, 'lambda_l2': 0.0, 'verbose': 1}
0.3588
0.9774344333333334

# Test

In [20]:
test = pd.read_csv("atec_anti_fraud_test_a.csv")
test.fillna(-1,inplace=True)

In [21]:
res = []
for i in range(NUM_DATASET):
    model = lgbmodellist[i]
    pred = model.predict(test[test.columns.difference(["id","label","weight","date"])],num_iteration = model.best_iteration)
    res.append(pred)

In [22]:
result = np.array(res)
result = result.mean(axis=0)
result = pd.DataFrame(result,columns=["score"])

In [23]:
sub = pd.DataFrame(test.id,columns=["id"])
sub = pd.concat([sub,result],axis=1)

In [24]:
sub.to_csv("submission62603.csv",index=False)